In [1]:
from collections import deque
import numpy as np
import grid2op

from grid2op.Runner import Runner
from grid2op.Converter import IdToAct
from grid2op.Agent.agentWithConverter import AgentWithConverter

In [2]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dense, subtract, add
from tensorflow.keras.layers import Input, Lambda

In [3]:
env_name = "rte_case14_realistic"
env = grid2op.make(env_name)

In [4]:
class TrainingParam(object):
    """
    A class to store the training parameters of the models. It was hard coded in the notebook 3.
    """
    def __init__(self,
                 DECAY_RATE=0.9,
                 BUFFER_SIZE=40000,
                 MINIBATCH_SIZE=64,
                 TOT_FRAME=3000000,
                 EPSILON_DECAY=10000,
                 MIN_OBSERVATION=50, #5000
                 FINAL_EPSILON=1/300,  # have on average 1 random action per scenario of approx 287 time steps
                 INITIAL_EPSILON=0.1,
                 TAU=0.01,
                 ALPHA=1,
                 NUM_FRAMES=1,
    ):
        print('TrainingParam __init__')
        self.DECAY_RATE = DECAY_RATE
        self.BUFFER_SIZE = BUFFER_SIZE
        self.MINIBATCH_SIZE = MINIBATCH_SIZE
        self.TOT_FRAME = TOT_FRAME
        self.EPSILON_DECAY = EPSILON_DECAY
        self.MIN_OBSERVATION = MIN_OBSERVATION   # 5000
        self.FINAL_EPSILON = FINAL_EPSILON  # have on average 1 random action per scenario of approx 287 time steps
        self.INITIAL_EPSILON = INITIAL_EPSILON
        self.TAU = TAU
        self.NUM_FRAMES = NUM_FRAMES
        self.ALPHA = ALPHA

In [5]:
class ReplayBuffer:
    """Constructs a buffer object that stores the past moves
    and samples a set of subsamples"""

    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.count = 0

    '''
    def add(self, s, a, r, d, s2):
        print('ReplayBuffer add')
        """Add an experience to the buffer"""
        # S represents current state, a is action,
        # r is reward, d is whether it is the end, 
        # and s2 is next state
        if np.any(~np.isfinite(s)) or np.any(~np.isfinite(s2)):
            # TODO proper handling of infinite values somewhere !!!!
            return

        experience = (s, a, r, d, s2)
        if self.count < self.buffer_size:
            self.buffer.append(experience)
            self.count += 1
        else:
            self.buffer.popleft()
            self.buffer.append(experience)

    def size(self):
        print('ReplayBuffer size')
        return self.count

    def sample(self, batch_size):
        print('ReplayBuffer sample')

        batch = []
        if self.count < batch_size:
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, batch_size)

        # Maps each experience in batch in batches of states, actions, rewards
        # and new states
        s_batch, a_batch, r_batch, d_batch, s2_batch = list(map(np.array, list(zip(*batch))))
        return s_batch, a_batch, r_batch, d_batch, s2_batch

    def clear(self):
        print('ReplayBuffer clear')
        self.buffer.clear()
        self.count = 0
    '''

In [6]:
class RLQvalue(object):
    """
    This class aims at representing the Q value (or more in case of SAC) parametrization by
    a neural network.

    It is composed of 2 different networks:
    - model: which is the main model
    - target_model: which has the same architecture and same initial weights as "model" but is updated less frequently
      to stabilize training

    It has basic methods to make predictions, to train the model, and train the target model.
    """
    def __init__(self, action_size, observation_size,
                 learning_rate=1e-5,
                 training_param=TrainingParam()):
        # TODO add more flexibilities when building the deep Q networks, with a "NNParam" for example.
        self.action_size = action_size
        self.observation_size = observation_size
        self.learning_rate_ = learning_rate
        self.qvalue_evolution = np.zeros((0,))
        self.training_param = training_param

        self.model = None
        self.target_model = None
    
    '''
    def construct_q_network(self):
        print('RLQvalue construct_q_network')
        raise NotImplementedError("Not implemented")
    '''

    def predict_movement(self, data, epsilon):
        """Predict movement of game controler where is epsilon
        probability randomly move."""
        rand_val = np.random.random(data.shape[0])
        print(f'>> rand_val = {rand_val}')
        q_actions = self.model.predict(data)
        print(f'>> q_actions = {q_actions}')
        opt_policy = np.argmax(np.abs(q_actions), axis=-1)
        print(f'>> argmax = {opt_policy}')
        opt_policy[rand_val < epsilon] = np.random.randint(0, self.action_size, size=(np.sum(rand_val < epsilon)))
        
        self.qvalue_evolution = np.concatenate((self.qvalue_evolution, q_actions[0, opt_policy]))
        print(f'>> qvalue_evolution = {self.qvalue_evolution}')
        #print(f'>> opt_policy = {opt_policy}')
        #print(f'>> q_actions = {q_actions[0, opt_policy]}')
        return opt_policy, q_actions[0, opt_policy]
    
    '''
    def train(self, s_batch, a_batch, r_batch, d_batch, s2_batch, observation_num):
        print('RLQvalue train')
        """Trains network to fit given parameters"""
        targets = self.model.predict(s_batch)
        fut_action = self.target_model.predict(s2_batch)
        targets[:, a_batch] = r_batch
        targets[d_batch, a_batch[d_batch]] += self.training_param.DECAY_RATE * np.max(fut_action[d_batch], axis=-1)

        loss = self.model.train_on_batch(s_batch, targets)
        # Print the loss every 100 iterations.
        if observation_num % 100 == 0:
            print("We had a loss equal to ", loss)
        return np.all(np.isfinite(loss))

    @staticmethod
    def _get_path_model(path, name=None):
        print('RLQvalue _get_path_model')
        if name is None:
            path_model = path
        else:
            path_model = os.path.join(path, name)
        path_target_model = "{}_target".format(path_model)
        return path_model, path_target_model

    def save_network(self, path, name=None, ext="h5"):
        print('RLQvalue save_network')
        # Saves model at specified path as h5 file
        # nothing has changed
        path_model, path_target_model = self._get_path_model(path, name)
        self.model.save('{}.{}'.format(path_model, ext))
        self.target_model.save('{}.{}'.format(path_target_model, ext))
        print("Successfully saved network.")

    
    def load_network(self, path, name=None, ext="h5"):
        print('RLQvalue load_network')
        # nothing has changed
        path_model, path_target_model = self._get_path_model(path, name)
        self.model = load_model('{}.{}'.format(path_model, ext))
        self.target_model = load_model('{}.{}'.format(path_target_model, ext))
        print("Succesfully loaded network.")

    def target_train(self):
        print('RLQvalue target_train')
        # nothing has changed from the original implementation
        model_weights = self.model.get_weights()
        target_model_weights = self.target_model.get_weights()
        for i in range(len(model_weights)):
            target_model_weights[i] = self.training_param.TAU * model_weights[i] + (1 - self.training_param.TAU) * \
                                      target_model_weights[i]
        self.target_model.set_weights(target_model_weights)
    '''

TrainingParam __init__


In [7]:
class DuelQ(RLQvalue):
    """Constructs the desired duelling deep q learning network"""
    def __init__(self, action_size, observation_size,
                 learning_rate=0.00001,
                 training_param=TrainingParam()):
        ## print('DuelQ __init__')
        RLQvalue.__init__(self, action_size, observation_size, learning_rate, training_param)
        self.construct_q_network()

    def construct_q_network(self):
        # Uses the network architecture found in DeepMind paper
        # The inputs and outputs size have changed, as well as replacing the convolution by dense layers.
        self.model = Sequential()
        
        input_layer = Input(shape=(self.observation_size*self.training_param.NUM_FRAMES,))
        
        lay1 = Dense(self.observation_size*self.training_param.NUM_FRAMES)(input_layer)
        lay1 = Activation('relu')(lay1)
        
        lay2 = Dense(self.observation_size)(lay1)
        lay2 = Activation('relu')(lay2)
        
        lay3 = Dense(2*self.action_size)(lay2)
        lay3 = Activation('relu')(lay3)
        
        fc1 = Dense(self.action_size)(lay3)
        advantage = Dense(self.action_size)(fc1)
        fc2 = Dense(self.action_size)(lay3)
        value = Dense(1)(fc2)
        
        meaner = Lambda(lambda x: K.mean(x, axis=1) )
        mn_ = meaner(advantage)
        tmp = subtract([advantage, mn_])
        policy = add([tmp, value])

        self.model = Model(inputs=[input_layer], outputs=[policy])
        self.model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate_))

        self.target_model = Model(inputs=[input_layer], outputs=[policy])
        self.target_model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate_))
        print("Successfully constructed networks.")

TrainingParam __init__


In [8]:
class MyDeepQAgent(AgentWithConverter):
    
    ## 1*0^-5 = 0.00001
    def __init__(self, action_space, mode="DDQN", learning_rate=1e-5, training_param=TrainingParam()):     
        
        ## Handle only vectors, and the type of action_space is GridObjects
        AgentWithConverter.__init__(self, action_space, action_space_converter=IdToAct)

        # and now back to the origin implementation
        self.replay_buffer = ReplayBuffer(training_param.BUFFER_SIZE)

        # compare to original implementation, i don't know the observation space size.
        # Because it depends on the component of the observation we want to look at. So these neural network will
        # be initialized the first time an observation is observe.
        self.deep_q = None
        self.mode = mode
        self.learning_rate = learning_rate
        self.training_param = training_param
    
    def convert_obs(self, observation):
        ## print(f'>> convert_obs = {np.concatenate((observation.rho, observation.line_status, observation.topo_vect))}')
        return np.concatenate((observation.rho, observation.line_status, observation.topo_vect))

    def my_act(self, transformed_observation, reward, done=False):
        ## print(f'>> transformed_observation = {transformed_observation}')
        if self.deep_q is None:
            self.init_deep_q(transformed_observation)
        
        predict_movement_int, *_ = self.deep_q.predict_movement(transformed_observation.reshape(1, -1), epsilon=0.0)
        print(f'>> predict_movement_int = {predict_movement_int}')
        print(*_)
        return int(predict_movement_int)

    def init_deep_q(self, transformed_observation):
        if self.deep_q is None:
            # the first time an observation is observed, I set up the neural network with the proper dimensions.
            if self.mode == "DQN":
                cls = DeepQ
            elif self.mode == "DDQN":
                cls = DuelQ
            elif self.mode == "SAC":
                cls = SAC
            else:
                raise RuntimeError("Unknown neural network named \"{}\". Supported types are \"DQN\", \"DDQN\" and "
                                   "\"SAC\"".format(self.mode))
            self.deep_q = cls(self.action_space.size(), observation_size=transformed_observation.shape[-1], learning_rate=self.learning_rate)
            print(f'>> action_size = {self.deep_q.action_size}, observation_size = {self.deep_q.observation_size}, learning_rate_ = {self.deep_q.learning_rate_}, qvalue_evolution = {self.deep_q.qvalue_evolution}, training_param = {self.deep_q.training_param}, model = {self.deep_q.model}, target_model = {self.deep_q.target_model}')
            print(f'>> self.deep_q = {self.deep_q}')
            
    '''
    def load_network(self, path):
        print('MyDeepQAgent load_network')
        # not modified compare to original implementation
        self.deep_q.load_network(path)
    '''

TrainingParam __init__


In [9]:
my_agent = MyDeepQAgent(env.action_space)

runner = Runner(**env.get_params_for_runner(), agentClass=MyDeepQAgent)
res = runner.run(nb_episode=1, max_iter=10)

Successfully constructed networks.
>> action_size = 451, observation_size = 96, learning_rate_ = 1e-05, qvalue_evolution = [], training_param = <__main__.TrainingParam object at 0x7fb9ee66fe20>, model = <keras.engine.functional.Functional object at 0x7fb9ef391b80>, target_model = <keras.engine.functional.Functional object at 0x7fb9eeff2850>
>> self.deep_q = <__main__.DuelQ object at 0x7fb9eefde5e0>
>> rand_val = [0.31292105]
1/1 [==============================] - 0s 85ms/step


2022-06-03 14:51:18.591893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


>> q_actions = [[-0.21707192 -0.3266574  -0.33051664 -0.0687148  -0.21444835 -0.22947392
  -0.42168558 -0.29451928 -0.55095696 -0.23684058 -0.14569393 -0.56864923
  -0.35897627 -0.08650753 -0.40262848 -0.3339302  -0.26194698 -0.35521877
  -0.43239993 -0.3855791  -0.38429844 -0.22488365 -0.2646321  -0.35977912
  -0.24205174 -0.39300185 -0.27291593 -0.33653688 -0.06619361 -0.38505754
  -0.38732278 -0.2057418  -0.3259206  -0.16456042 -0.14095753 -0.5245169
  -0.386196   -0.2889656  -0.5852301  -0.24179494 -0.4951021  -0.40860963
  -0.40560594 -0.2738433  -0.46450445 -0.28493422 -0.4076653  -0.36678803
  -0.17506441 -0.41046613 -0.29346982 -0.3597146  -0.3254201  -0.32701325
  -0.41525978 -0.3849613  -0.33242655 -0.4870559  -0.2728156  -0.36278653
  -0.3980261  -0.39003712 -0.22230092 -0.23223585 -0.46817344 -0.209116
  -0.43943396 -0.31179953 -0.40656173 -0.29915306 -0.31598338 -0.20944278
  -0.10257858 -0.20699224 -0.43247244 -0.26575372 -0.3611637  -0.2173027
  -0.49998438 -0.49821424 -

In [10]:
for _, chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics with id {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

	For chronics with id 000
		 - cumulative reward: -10.000000
		 - number of time steps completed: 1 / 10
